In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
import time

In [3]:

df = pd.read_excel('C:/Users/Hernán Ifrán/Downloads/RECLAMOSFINAL.xlsx')
df.dropna(subset=['problema_n', 'obsitem'], inplace=True)

In [4]:

df['problema_n'] = df['problema_n'].str.replace('K - Fallas en encendido electrónico y/o luz de horno', 'K - Fallas en encendido electrónico')
df['problema_n'] = df['problema_n'].str.replace('B - Pérdidas de gas sin/con piezas quemadas', 'B - Pérdidas de gas – sin/con piezas quemadas')
df['problema_n'] = df['problema_n'].str.replace('J - Defectos estéticos - Modelos Ac.  inoxidable', 'I - Defectos estéticos')
df['problema_n'] = df['problema_n'].str.replace('C1 - Horno No enciende', 'C2 - Horno Mal funcionamiento')
df['problema_n'] = df['problema_n'].str.replace('C3 - Horno Se Apaga', 'C2 - Horno Mal funcionamiento')
df['problema_n'] = df['problema_n'].str.replace('E - El agua sale con temperatura baja, alta', 'E - El agua sale con temperatura baja / alta')


df['problema_n'] = df['problema_n'].str.replace('D4 - Otra Hornalla no enciende', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].str.replace('D6 - Otra Hornalla Se Apaga', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].str.replace('D3 - Hornalla Se Apaga', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].str.replace('D1 - Hornalla No enciende', 'D2 - Hornalla Mal funcionamiento')
df['problema_n'] = df['problema_n'].str.replace('D5 - Otra Hornalla Mal funcionamiento', 'D2 - Hornalla Mal funcionamiento')

C:\Users\Hernán Ifrán\AppData\Local\Temp\ipykernel_18256\400429262.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['problema_n'] = df['problema_n'].str.replace('J - Defectos estéticos - Modelos Ac.  inoxidable', 'I - Defectos estéticos')


In [5]:

condicion = ~df['problema_n'].isin(['K - Otro problema no mencionado en opciones anteriores', 'J - Otras piezas en mal estado','E - No se apaga el calefón al cerrar la circulación de agua','L - Falta/Falla accesorios Kit MULTIGAS','Q - Problema no mencionado en opciones anteriores','P - Otras piezas en mal estado','H - Piezas se caen, mal fijadas o mal posicionadas','N - Manija rota','H - Accesorios faltantes','I - Otros defectos','G - Piezas faltantes / mal fijadas','F - No apaga','I - Accesorios cambio de GAS','H - Accesorios faltantes','J - Defectos estéticos','G - Piezas faltantes / mal fijadas','K - Problema no mencionado en opciones anteriores'])
df = df[condicion]



df['Concaobsitem'] = df['descripcion'].str.cat([df['alias_8_nombre3'], df['obsitem']], sep=' ')
df['Concaobsitem'] = df['Concaobsitem'].astype(str)

In [6]:
descripciones = df['Concaobsitem'].tolist()
etiquetas = df['problema_n'].tolist()
label_encoder = LabelEncoder()
etiquetas_numericas = label_encoder.fit_transform(etiquetas)

In [7]:

stemmer = SnowballStemmer('spanish')
stop_words = set(stopwords.words('spanish'))

def preprocesar_texto(texto):
    palabras = nltk.word_tokenize(texto.lower())
    palabras = [stemmer.stem(palabra) for palabra in palabras if palabra.isalpha() and palabra not in stop_words]
    return ' '.join(palabras)

descripciones_preprocesadas = [preprocesar_texto(desc) for desc in descripciones]

In [8]:

X_train, X_test, y_train, y_test = train_test_split(descripciones_preprocesadas, etiquetas_numericas, test_size=0.2, random_state=100201)


In [9]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  
    ('clf', SVC(kernel='linear'))
])


pipeline.fit(X_train, y_train)
scores = cross_val_score(pipeline, X_train, y_train, cv=5)  

print("Accuracy en validación cruzada:", scores.mean())

C:\Users\Hernán Ifrán\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Accuracy en validación cruzada: 0.7419639658125559


In [10]:

y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred, labels=np.unique(etiquetas_numericas), target_names=label_encoder.inverse_transform(np.unique(etiquetas_numericas)), zero_division=1)
print(report)


                                                                                                   precision    recall  f1-score   support

                                                                    B - Descargas de electricidad       0.33      0.15      0.21        20
                                               B - La llama no enciende al abrir el grifo de agua       0.78      0.97      0.86        33
                                                    B - Pérdidas de gas – sin/con piezas quemadas       0.40      0.11      0.17        19
                                                          B1 - Pérdidas de gas con piezas dañadas       0.33      0.12      0.18        67
                                                          B2 - Pérdidas de gas sin piezas dañadas       0.67      0.71      0.69       195
                              C - La llama se apaga luego de algunos minutos de estar funcionando       0.93      0.76      0.84        17
                          

In [12]:

archivo_evaluacion = 'C:/Users/Hernán Ifrán/Downloads/testesco.txt'  
with open(archivo_evaluacion, 'r', encoding='latin-1') as file:
    lineas = file.readlines()

resultados = []

for linea in lineas:
    problema = linea.strip()
    problema_preprocesado = preprocesar_texto(problema)
    predicciones = pipeline.predict([problema_preprocesado])
    clase_predicha = label_encoder.inverse_transform(predicciones)[0]
    resultados.append({
        'Problema': problema,
        'Etiqueta de Reclamo': clase_predicha
    })

resultados_df = pd.DataFrame(resultados)

archivo_resultado = 'C:/Users/Hernán Ifrán/Downloads/testresultado.txt'
resultados_df.to_csv(archivo_resultado, index=False, sep='\t', encoding='utf-8')

print("Resultados guardados en:", archivo_resultado)




Resultados guardados en: C:/Users/Hernán Ifrán/Downloads/testresultado.txt
